<a href="https://colab.research.google.com/github/roshancharlie/Mini-Imdb-Application/blob/main/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing The Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
netflix = pd.read_csv('https://raw.githubusercontent.com/roshancharlie/Mini-Imdb-Application/main/dataset/netflix_titles.csv')
amazon = pd.read_csv('https://raw.githubusercontent.com/roshancharlie/Mini-Imdb-Application/main/dataset/amazon_prime_titles.csv')
hotstar = pd.read_csv('https://raw.githubusercontent.com/roshancharlie/Mini-Imdb-Application/main/dataset/disney_plus_titles.csv')

In [3]:
data = pd.concat([netflix,amazon,hotstar])
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19925 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       19925 non-null  object
 1   type          19925 non-null  object
 2   title         19925 non-null  object
 3   director      14736 non-null  object
 4   cast          17677 non-null  object
 5   country       9879 non-null   object
 6   date_added    10399 non-null  object
 7   release_year  19925 non-null  int64 
 8   rating        19581 non-null  object
 9   duration      19922 non-null  object
 10  listed_in     19925 non-null  object
 11  description   19925 non-null  object
dtypes: int64(1), object(11)
memory usage: 2.0+ MB


The dataset contains several missing values, and in order to fill them, we will utilize web scraping and the IMDb API.

# Importing the Dataset with Imputed Values

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/roshancharlie/Mini-Imdb-Application/main/dataset/final_movie_shows.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20821 entries, 0 to 20820
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       20821 non-null  object
 1   type          20821 non-null  object
 2   title         20821 non-null  object
 3   director      16439 non-null  object
 4   cast          20186 non-null  object
 5   date_added    10896 non-null  object
 6   release_year  20821 non-null  int64 
 7   rating        20465 non-null  object
 8   duration      20818 non-null  object
 9   listed_in     20821 non-null  object
 10  description   20821 non-null  object
 11  country_y     19579 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.9+ MB


# Data Exploration and Data Preparation

In [7]:
df['type'].unique()

array(['Movie', 'TV Show'], dtype=object)

In [8]:
df['title'].unique()

array(['The Grand Seduction', 'Take Care Good Night',
       'Secrets of Deception', ..., "Marvel's Rocket & Groot (Shorts)",
       'Spider-Man (Shorts)', 'Star Wars Rebels (Shorts)'], dtype=object)

In [9]:
df['director'].unique()

array(['Don McKellar', 'Girish Joshi', 'Josh Webber', ..., 'Mike Jacobs',
       'Hart Bochner', 'Ben Fast'], dtype=object)

In [10]:
df['cast'].unique()

array(['Brendan Gleeson, Taylor Kitsch, Gordon Pinsent',
       'Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar',
       'Tom Sizemore, Lorenzo Lamas, Robert LaSardo, Richard Jones, Yancey Arias, Noel Gugliemi',
       ...,
       "['Chukwudi Iwuji', 'Bradley Cooper', 'Pom Klementieff', 'Dave Bautista', 'Karen Gillan', 'Vin Diesel', 'Austin Freeman', 'Stephen Blackehart', 'Terence Rosemore', 'Maria Bakalova', 'Sean Gunn', 'Sarah Alami', 'Jasmine Munoz', 'Chris Pratt', 'Giovannie Cruz', 'Will Poulter', 'Nico Santos', 'Miriam Shor', 'Noa Raskin', 'Linda Cardellini', 'Asim Chaudhry', 'Mikaela Hoover', 'Elizabeth Debicki', 'Judy Greer', 'Reinaldo Faberlle', 'Sylvester Stallone', 'Tara Strong', 'Jared Gore', 'Michael Rosenbaum', 'Elan Gale', 'Molly C. Quinn', 'Gerardo Davila', 'Dee Bradley Baker', 'Zoe Saldana', 'Hanna Pak', 'Jennifer Holland', 'Nathan Fillion', 'Kyle Mclean', 'Benjamin Byron Davis', 'Tiffany Smith', 'Joe Daru', 'Daniela Melchior', 'Jonathan Mercedes', 'Jonathan Fritschi'

In [11]:
df['release_year'].unique()

array([2014, 2018, 2017, 1989, 2016, 1994, 2020, 2019, 1996, 2011, 1991,
       2008, 2012, 2002, 1955, 2001, 1995, 2013, 1974, 1963, 1987, 1956,
       2004, 2015, 1939, 2007, 2021, 2009, 1997, 1951, 1979, 1971, 1992,
       2006, 1988, 1975, 1946, 2005, 2003, 1968, 2010, 1981, 1984, 1993,
       1973, 1998, 1986, 1990, 2000, 1967, 1999, 1959, 1962, 1978, 1983,
       1982, 1980, 1961, 1985, 1976, 1972, 1964, 1954, 1958, 1970, 1960,
       1966, 1969, 1977, 1965, 1945, 1942, 1947, 1944, 1957, 1950, 1952,
       1948, 1936, 1935, 1940, 1937, 1938, 1953, 1943, 1928, 1949, 1941,
       1934, 1933, 1920, 1932, 1929, 1925, 1924, 1922, 1926, 1930, 1931,
       1923, 1927])

In [12]:
df['rating'].unique()

array([nan, '13+', '18+', 'ALL', 'PG-13', 'R', 'G', 'NR', 'PG', '16+',
       '7+', 'NOT_RATE', 'TV-MA', 'TV-14', 'TV-PG', 'TV-Y7', 'TV-G',
       'TV-Y', 'NC-17', '74 min', '84 min', '66 min', 'TV-Y7-FV', 'UR',
       'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES', 'TV-NR'],
      dtype=object)

In [13]:
df['duration'].unique()

array(['113 min', '110 min', '74 min', '69 min', '45 min', '52 min',
       '98 min', '131 min', '87 min', '92 min', '88 min', '93 min',
       '94 min', '46 min', '96 min', '64 min', '138 min', '90 min',
       '151 min', '149 min', '125 min', '121 min', '102 min', '107 min',
       '126 min', '142 min', '104 min', '106 min', '159 min', '116 min',
       '91 min', '99 min', '61 min', '82 min', '80 min', '89 min',
       '77 min', '81 min', '100 min', '101 min', '103 min', '146 min',
       '27 min', '132 min', '157 min', '85 min', '127 min', '44 min',
       '133 min', '148 min', '119 min', '124 min', '174 min', '137 min',
       '68 min', '78 min', '75 min', '134 min', '163 min', '162 min',
       '112 min', '114 min', '147 min', '58 min', '139 min', '111 min',
       '55 min', '120 min', '136 min', '129 min', '86 min', '153 min',
       '166 min', '118 min', '84 min', '135 min', '109 min', '117 min',
       '145 min', '53 min', '141 min', '59 min', '167 min', '105 min',
       '83 m

In [14]:
df['listed_in'].unique()

array(['Comedy, Drama', 'Drama, International', 'Action, Drama, Suspense',
       ..., 'Anthology, Documentary', 'Animals & Nature, Kids, Music',
       'Comedy, Family, Game Show / Competition'], dtype=object)

In [15]:
df['description'][0]

"A small fishing village must procure a local doctor to secure a lucrative business contract. When unlikely candidate and big city doctor Paul Lewis lands in their lap for a trial residence, the townsfolk rally together to charm him into staying. As the doctor's time in the village winds to a close, acting mayor Murray French has no choice but to pull out all the stops."

In [16]:
df['country_y'].unique()

array(['Canada', 'India', 'United States', 'United Kingdom', 'France',
       'Spain', 'United Kingdom, France', 'United States, Italy',
       'United States, India', 'United States, United Kingdom, Germany',
       'China, United States, United Kingdom',
       'Denmark, United Kingdom, Czech Republic, Netherlands',
       'United States, Ireland', 'United Kingdom, United States, India',
       'United Kingdom, India', 'United States, China',
       'United States, Thailand', 'Brazil, United States, India',
       'Pakistan', 'Germany', 'United Kingdom, United States',
       'Hong Kong, China', 'Belgium', 'Italy', 'Netherlands',
       'Canada, United States', 'United States, United Kingdom',
       'United States, Portugal', 'United States, France',
       'Canada, United States, Brazil', 'France, United States',
       'India, United States', 'India, Sweden',
       'United Kingdom, Hungary, United States',
       'United Kingdom, France, United States',
       'United States, Fra

In [17]:
df['cast']

0           Brendan Gleeson, Taylor Kitsch, Gordon Pinsent
1         Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar
2        Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...
3        Interviews with: Pink, Adele, Beyoncé, Britney...
4        Harry Dean Stanton, Kieran O'Brien, George Cos...
                               ...                        
20816    ['Kathryn Newton', 'Jonathan Majors', 'Chris H...
20817    ['Chukwudi Iwuji', 'Bradley Cooper', 'Pom Klem...
20818        ['Kevin Michael Richardson', 'Trevor Devall']
20819                                                  NaN
20820    ['Dave Filoni', 'Taylor Gray', 'Freddie Prinze...
Name: cast, Length: 20821, dtype: object

In [18]:
df['cast'] = df['cast'].str.split(',').str[:3]
df['cast'] = df['cast'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

In [19]:
df['cast']

0         Brendan Gleeson,  Taylor Kitsch,  Gordon Pinsent
1        Mahesh Manjrekar,  Abhay Mahajan,  Sachin Khed...
2            Tom Sizemore,  Lorenzo Lamas,  Robert LaSardo
3                  Interviews with: Pink,  Adele,  Beyoncé
4        Harry Dean Stanton,  Kieran O'Brien,  George C...
                               ...                        
20816    ['Kathryn Newton',  'Jonathan Majors',  'Chris...
20817    ['Chukwudi Iwuji',  'Bradley Cooper',  'Pom Kl...
20818       ['Kevin Michael Richardson',  'Trevor Devall']
20819                                                     
20820    ['Dave Filoni',  'Taylor Gray',  'Freddie Prin...
Name: cast, Length: 20821, dtype: object

In [20]:
def extract_names(names):
    if isinstance(names, list):
        return ', '.join(names[:3])
    elif isinstance(names, str):
        names_list = [name.strip() for name in names.strip("[]").split(',') if name.strip()]
        return ', '.join(names_list[:3])
    else:
        return names

In [21]:
df['cast'] = df['cast'].apply(extract_names)

In [22]:
df['cast']

0           Brendan Gleeson, Taylor Kitsch, Gordon Pinsent
1         Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar
2              Tom Sizemore, Lorenzo Lamas, Robert LaSardo
3                    Interviews with: Pink, Adele, Beyoncé
4        Harry Dean Stanton, Kieran O'Brien, George Cos...
                               ...                        
20816    'Kathryn Newton', 'Jonathan Majors', 'Chris He...
20817    'Chukwudi Iwuji', 'Bradley Cooper', 'Pom Kleme...
20818          'Kevin Michael Richardson', 'Trevor Devall'
20819                                                     
20820    'Dave Filoni', 'Taylor Gray', 'Freddie Prinze ...
Name: cast, Length: 20821, dtype: object

In [23]:
def format_names(names):
    if isinstance(names, str):
        names_list = [name.strip().strip("'") for name in names.split(",") if name.strip()]
        return ", ".join(names_list)
    else:
        return names

In [24]:
df['cast'] = df['cast'].apply(format_names)

In [25]:
df['cast'][20816]

'Kathryn Newton, Jonathan Majors, Chris Hemsworth'

In [26]:
df['cast'].unique()

array(['Brendan Gleeson, Taylor Kitsch, Gordon Pinsent',
       'Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar',
       'Tom Sizemore, Lorenzo Lamas, Robert LaSardo', ...,
       'Chukwudi Iwuji, Bradley Cooper, Pom Klementieff',
       'Kevin Michael Richardson, Trevor Devall',
       'Dave Filoni, Taylor Gray, Freddie Prinze Jr.'], dtype=object)

In [27]:
df.isnull().sum()

show_id            0
type               0
title              0
director        4382
cast               0
date_added      9925
release_year       0
rating           356
duration           3
listed_in          0
description        0
country_y       1242
dtype: int64

In [28]:
df.drop(['director','date_added','show_id','duration'],axis=1,inplace=True)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20821 entries, 0 to 20820
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          20821 non-null  object
 1   title         20821 non-null  object
 2   cast          20821 non-null  object
 3   release_year  20821 non-null  int64 
 4   rating        20465 non-null  object
 5   listed_in     20821 non-null  object
 6   description   20821 non-null  object
 7   country_y     19579 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.3+ MB


In [30]:
df.dropna(inplace=True)

In [31]:
df.reset_index(inplace=True, drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19250 entries, 0 to 19249
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          19250 non-null  object
 1   title         19250 non-null  object
 2   cast          19250 non-null  object
 3   release_year  19250 non-null  int64 
 4   rating        19250 non-null  object
 5   listed_in     19250 non-null  object
 6   description   19250 non-null  object
 7   country_y     19250 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.2+ MB


In [32]:
df['country_y']

0                         India
1                         India
2                         India
3        United Kingdom, France
4                 United States
                  ...          
19245             United States
19246             United States
19247             United States
19248             United States
19249             United States
Name: country_y, Length: 19250, dtype: object

In [33]:
df['country_y'] = df['country_y'].str.split(',').str[0]

In [34]:
df['country_y'].unique()

array(['India', 'United Kingdom', 'United States', 'China', 'Denmark',
       'Canada', 'Brazil', 'Pakistan', 'Germany', 'Hong Kong', 'Belgium',
       'Italy', 'Netherlands', 'France', 'Spain', 'Ireland',
       'South Africa', 'Australia', 'Egypt', 'Switzerland', 'Afghanistan',
       'Kosovo', 'Austria', 'Malaysia', 'Poland', 'Japan', 'Nigeria',
       'Argentina', 'New Zealand', 'Colombia', 'Mexico', 'Taiwan',
       'Bulgaria', '', 'Saudi Arabia', 'Thailand', 'Indonesia', 'Kuwait',
       'South Korea', 'Vietnam', 'Lebanon', 'Romania', 'Philippines',
       'United Arab Emirates', 'Sweden', 'Syria', 'Mauritius', 'Turkey',
       'Czech Republic', 'Cameroon', 'Russia', 'Kenya', 'Chile',
       'Bangladesh', 'Portugal', 'Hungary', 'Norway',
       'CanadaUnited KingdomUnited States[1]', 'Singapore', 'Iceland',
       'Serbia', 'Namibia', 'Uruguay', 'Peru', 'Mozambique', 'Ghana',
       'Zimbabwe', 'Israel', 'Paraguay', 'Ukraine', 'Cambodia',
       'Soviet Union', 'Georgia', 'Iran',

In [35]:
replacements = [
    ('CanadaUnited KingdomUnited States[1]', 'Canada'),
    ('CanadaFranceSouth Africa', 'Canada'),
    ('CanadaUnited Kingdom', 'Canada'),
    ('CanadaFrance', 'Canada'),
    ('United KingdomUnited States', 'United Kingdom'),
    ('Italy\nUnited States (revived series)[1][2]', 'Italy'),
    ('United StatesIreland', 'United States'),
    ('France\nUnited States', 'France'),
    ('United Kingdom[1]', 'United Kingdom'),
    ('FranceCanada (season 3–5)', 'France'),
    ('Sweden\nUnited States', 'Sweden'),
    ('Original series:Switzerland\nRevival series:United Kingdom', 'Switzerland'),
    ('United Kingdom (seasons 3-4)', 'United Kingdom'),
    ('United States & Canada', 'United States'),
    ('CanadaUnited Kingdom', 'Canada'),
    ('Canada\nUnited States', 'Canada'),
    ('CanadaHong Kong', 'Canada'),
    ('SpainItaly (Season 1)Philippines (Season 1)Ireland (Season 2)India (Season 2)', 'Spain'),
    ('Canada\nChina', 'Canada'),
    ('United Kingdom / US', 'United Kingdom'),
    ('United KingdomSingapore', 'United Kingdom'),
    ('CanadaLuxembourg', 'Canada'),
    ('JapanCanadaAustralia', 'Japan'),
    ('Australia\nCanada', 'Australia'),
    ('United States\nIreland', 'United States'),
    ('Republic of China (Taiwan)', 'Taiwan'),
    ('SingaporeChina', 'Singapore'),
    ('MalaysiaSingapore', 'Malaysia'),
    ('United StatesJapan', 'United States'),
    ('United StatesFrance', 'United States'),
    ('United States\nTaiwan[1]', 'United States'),
    ('JapanCanada', 'Japan'),
    ('Belarus (2015-22)Poland (2022-)', 'Belarus'),
    ('South KoreaChinaCanada[citation needed]', 'South Korea'),
    ('United States (Full Series)[1]', 'United States'),
    ('FranceCanada', 'France'),
    ('France\nCanada (season 3–5)', 'France'),
    ('England', 'United Kingdom'),
    ('Dominican Republic', 'Dominican Republic'),
    ('Bahrain', 'Bahrain'),
    ('Various participating countries', 'Various'),
    ('Asia', 'Asia'),
    ('Liechtenstein', 'Liechtenstein'),
    ('United States[1]', 'United States'),
    ('USA', 'United States'),
    ('CanadaUnited KingdomUnited States[1]', 'Canada'),
    ('Canada\nAustralia\nUnited States', 'Canada'),
    ('United States\nCanada', 'United States'),
    ('United States\nItaly', 'United States'),
    ('United StatesNew Zealand', 'United States'),
    ('United Kingdom[1]', 'United Kingdom'),
    ('United StatesCanada', 'United States'),
    ('United States\nFrance', 'United States'),
    ('United StatesUnited Kingdom', 'United States'),
    ('United KingdomCanada (seasons 3-4)', 'United Kingdom'),
    ('United States\nSingapore\nCanada', 'United States'),
    ('United StatesUnited Kingdom (Full Series)[1]', 'United States'),
    ('South KoreaChinaCanada[citation needed]', 'South Korea'),
    ('South Korea\nChina\nUnited States', 'South Korea'),
    ('United States\nUnited Kingdom\nAustria', 'United States'),
    ('United Kingdom\nUnited States', 'United Kingdom'),
    ('CanadaUnited States', 'Canada'),
    ('South KoreaSingapore', 'South Korea'),
    ('AustraliaUnited States', 'Australia'),
    ('United States\nSingapore\nCanada', 'United States'),
    ('United KingdomCanada', 'United Kingdom'),
    ('USA & Canada', 'United States'),
    ('United States\nCanada\nTaiwan[1]', 'United States'),
    ('JapanCanadaUnited States', 'Japan'),
    ('United States\nCanada\nTaiwan[1]', 'United States'),
    ('Brazil\nUnited Kingdom', 'Brazil'),
    ('JapanCanadaUnited StatesAustralia','Japan'),
    ('United StatesCanada[1]','United States')]

replacements_dict = dict(replacements)
df['country_y'] = df['country_y'].map(replacements_dict).fillna(df['country_y'])

In [36]:
df['country_y'].unique()

array(['India', 'United Kingdom', 'United States', 'China', 'Denmark',
       'Canada', 'Brazil', 'Pakistan', 'Germany', 'Hong Kong', 'Belgium',
       'Italy', 'Netherlands', 'France', 'Spain', 'Ireland',
       'South Africa', 'Australia', 'Egypt', 'Switzerland', 'Afghanistan',
       'Kosovo', 'Austria', 'Malaysia', 'Poland', 'Japan', 'Nigeria',
       'Argentina', 'New Zealand', 'Colombia', 'Mexico', 'Taiwan',
       'Bulgaria', '', 'Saudi Arabia', 'Thailand', 'Indonesia', 'Kuwait',
       'South Korea', 'Vietnam', 'Lebanon', 'Romania', 'Philippines',
       'United Arab Emirates', 'Sweden', 'Syria', 'Mauritius', 'Turkey',
       'Czech Republic', 'Cameroon', 'Russia', 'Kenya', 'Chile',
       'Bangladesh', 'Portugal', 'Hungary', 'Norway', 'Singapore',
       'Iceland', 'Serbia', 'Namibia', 'Uruguay', 'Peru', 'Mozambique',
       'Ghana', 'Zimbabwe', 'Israel', 'Paraguay', 'Ukraine', 'Cambodia',
       'Soviet Union', 'Georgia', 'Iran', 'Finland', 'Venezuela',
       'Slovenia', 'Gu

In [37]:
df

,type,title,cast,release_year,rating,listed_in,description,country_y
0,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,13+,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,India
1,Movie,Whoop!,Abish Mathew,2018,18+,"Arts, Entertainment, and Culture",Abish Mathew is the world's greatest stand-up ...,India
2,Movie,Wedding Cha Shinema,"Shivraj Waichal, Rucha Inamdar, Mukta Barve",2019,ALL,"Comedy, International",An aspiring filmmaker reluctantly takes up the...,India
3,Movie,Waiting for You,"Colin Morgan, Fanny Ardant, Abdelkrim Bahloul",2018,13+,"Drama, Romance, Suspense",Paul (Morgan) investigates his late father's i...,United Kingdom
4,Movie,Vows of Deception,"Cheryl Ladd, Nick Manusco, Nancy Cartwright",1996,PG-13,"Drama, Suspense","When a prominent lawyer is found murdered, his...",United States
...,...,...,...,...,...,...,...,...
19245,Movie,Marvel Studios: Expanding the Universe,"Stephen Broussard, Kevin Feige, Mary Livanos",2019,TV-PG,"Documentary, Family",An exciting look into the future of Marvel Stu...,United States
19246,TV Show,Marvel's Avengers: Secret Wars (Shorts),"Kathryn Newton, Jonathan Majors, Chris Hemsworth",2016,TV-Y7,"Action-Adventure, Animation, Kids",A new team of heroes joins the Avengers.,United States
19247,TV Show,Marvel's Guardians of the Galaxy (Shorts),"Chukwudi Iwuji, Bradley Cooper, Pom Klementieff",2014,TV-Y7,"Action-Adventure, Animation, Kids",The stories of five outlaws who became the Gua...,United States
19248,TV Show,Marvel's Rocket & Groot (Shorts),"Kevin Michael Richardson, Trevor Devall",2016,TV-Y7,"Action-Adventure, Animation, Buddy",Rocket and Groot shop for a new ship.,United States


In [38]:
df['listed_in'].unique()

array(['Drama, International', 'Arts, Entertainment, and Culture',
       'Comedy, International', ..., 'Anthology, Documentary',
       'Animals & Nature, Kids, Music',
       'Comedy, Family, Game Show / Competition'], dtype=object)

In [39]:
import re
df['listed_in'] = df['listed_in'].apply(lambda x: re.sub('[^a-zA-Z]+', ',', x))
df['listed_in'] = df['listed_in'].replace(',and','')

In [40]:
df['listed_in']

0                     Drama,International
1          Arts,Entertainment,and,Culture
2                    Comedy,International
3                  Drama,Romance,Suspense
4                          Drama,Suspense
                       ...               
19245                  Documentary,Family
19246     Action,Adventure,Animation,Kids
19247     Action,Adventure,Animation,Kids
19248    Action,Adventure,Animation,Buddy
19249     Action,Adventure,Animation,Kids
Name: listed_in, Length: 19250, dtype: object

In [41]:
df.duplicated().value_counts()

False    18548
True       702
dtype: int64

In [42]:
df.drop_duplicates(inplace=True)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18548 entries, 0 to 19249
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          18548 non-null  object
 1   title         18548 non-null  object
 2   cast          18548 non-null  object
 3   release_year  18548 non-null  int64 
 4   rating        18548 non-null  object
 5   listed_in     18548 non-null  object
 6   description   18548 non-null  object
 7   country_y     18548 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.3+ MB


In [44]:
df.head(10)

,type,title,cast,release_year,rating,listed_in,description,country_y
0,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,13+,"Drama,International",A Metro Family decides to fight a Cyber Crimin...,India
1,Movie,Whoop!,Abish Mathew,2018,18+,"Arts,Entertainment,and,Culture",Abish Mathew is the world's greatest stand-up ...,India
2,Movie,Wedding Cha Shinema,"Shivraj Waichal, Rucha Inamdar, Mukta Barve",2019,ALL,"Comedy,International",An aspiring filmmaker reluctantly takes up the...,India
3,Movie,Waiting for You,"Colin Morgan, Fanny Ardant, Abdelkrim Bahloul",2018,13+,"Drama,Romance,Suspense",Paul (Morgan) investigates his late father's i...,United Kingdom
4,Movie,Vows of Deception,"Cheryl Ladd, Nick Manusco, Nancy Cartwright",1996,PG-13,"Drama,Suspense","When a prominent lawyer is found murdered, his...",United States
5,Movie,Voice From the Stone,"Emilia Clarke, Marton Csokas, Caterina Murino",2017,R,"Drama,Suspense","Set in 1950s Tuscany, Voice from the Stone is ...",United States
6,Movie,Viswasam,"Ajith Kumar, Nayanthara",2019,13+,"Action,Drama,International","A village ruffian, who settles disputes in his...",India
7,Movie,Virus,"Revathy, Kunchako Boban, Parvathy Thiruvoth",2019,ALL,"Drama,Science,Fiction,Suspense","Virus is a fiction based on true events, revol...",India
8,Movie,Vicky Velingkar,"Sonalee Kulkarni, Spruha Joshi, Sangram Samel",2019,13+,"International,Suspense",Vicky Velingkar is a struggling comic book art...,India
9,Movie,Vellai Pookal,"Vivek, Charlie, Dev",2019,13+,"International,Suspense",Vellai Pookal is a thriller based on the story...,United States


In [45]:
df[df['rating'].str.contains('min')]

,type,title,cast,release_year,rating,listed_in,description,country_y
3744,Movie,Louis C.K. 2017,Louis C.K.,2017,74 min,Movies,"Louis C.K. muses on religion, eternal love, gi...",United States
3910,Movie,Louis C.K.: Hilarious,Louis C.K.,2010,84 min,Movies,Emmy-winning comedy writer Louis C.K. brings h...,United States
3915,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,2015,66 min,Movies,The comic puts his trademark hilarious/thought...,United States


In [46]:
rating_categories = {'13+': 'PG-13','18+': 'R','ALL': 'G','PG-13': 'PG-13','R': 'R','G': 'G','NR': 'R','PG': 'PG','16+': 'PG-13','7+': 'G',
    'NOT_RATE': 'NR','TV-MA': 'NC-17','TV-14': 'R','TV-PG': 'PG','TV-Y7': 'G','TV-G': 'G','TV-Y': 'G','NC-17': 'NC-17','74 min': 'NR','84 min': 'NR',
    '66 min': 'NR','TV-Y7-FV': 'G','UR': 'NR','UNRATED': 'NR','AGES_16_': 'PG-13','AGES_18_': 'R','TV-NR': 'NR' }
df['rating'] = df['rating'].map(rating_categories)

In [47]:
df['rating'].unique()

array(['PG-13', 'R', 'G', 'PG', 'NR', 'NC-17'], dtype=object)

In [48]:
df['description'] = df['description'].str.lower()

In [49]:
df['description'][0]

'a metro family decides to fight a cyber criminal threatening their stability and pride.'

In [50]:
df.head()

,type,title,cast,release_year,rating,listed_in,description,country_y
0,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,PG-13,"Drama,International",a metro family decides to fight a cyber crimin...,India
1,Movie,Whoop!,Abish Mathew,2018,R,"Arts,Entertainment,and,Culture",abish mathew is the world's greatest stand-up ...,India
2,Movie,Wedding Cha Shinema,"Shivraj Waichal, Rucha Inamdar, Mukta Barve",2019,G,"Comedy,International",an aspiring filmmaker reluctantly takes up the...,India
3,Movie,Waiting for You,"Colin Morgan, Fanny Ardant, Abdelkrim Bahloul",2018,PG-13,"Drama,Romance,Suspense",paul (morgan) investigates his late father's i...,United Kingdom
4,Movie,Vows of Deception,"Cheryl Ladd, Nick Manusco, Nancy Cartwright",1996,PG-13,"Drama,Suspense","when a prominent lawyer is found murdered, his...",United States


In [51]:
df['description'] = df['description'].apply(lambda x:x.split())

In [52]:
df.head()

,type,title,cast,release_year,rating,listed_in,description,country_y
0,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,PG-13,"Drama,International","[a, metro, family, decides, to, fight, a, cybe...",India
1,Movie,Whoop!,Abish Mathew,2018,R,"Arts,Entertainment,and,Culture","[abish, mathew, is, the, world's, greatest, st...",India
2,Movie,Wedding Cha Shinema,"Shivraj Waichal, Rucha Inamdar, Mukta Barve",2019,G,"Comedy,International","[an, aspiring, filmmaker, reluctantly, takes, ...",India
3,Movie,Waiting for You,"Colin Morgan, Fanny Ardant, Abdelkrim Bahloul",2018,PG-13,"Drama,Romance,Suspense","[paul, (morgan), investigates, his, late, fath...",United Kingdom
4,Movie,Vows of Deception,"Cheryl Ladd, Nick Manusco, Nancy Cartwright",1996,PG-13,"Drama,Suspense","[when, a, prominent, lawyer, is, found, murder...",United States


In [53]:
df.release_year.value_counts().head(50)

2021    1956
2020    1824
2019    1805
2018    1706
2017    1555
2016    1393
2015     919
2014     738
2013     572
2012     494
2011     454
2010     398
2009     335
2008     330
2007     267
2006     260
2005     217
2004     193
2003     169
2002     161
2000     145
2001     145
1999     128
1998     120
1997     113
1995     105
1996     103
1994      98
1993      95
1992      87
1991      69
1990      66
1988      66
1989      60
1986      59
1987      48
1985      46
1984      44
1980      38
1981      36
1974      35
1975      35
1936      34
1982      34
1979      31
1978      31
1937      30
1941      28
1972      28
1973      28
Name: release_year, dtype: int64

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18548 entries, 0 to 19249
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          18548 non-null  object
 1   title         18548 non-null  object
 2   cast          18548 non-null  object
 3   release_year  18548 non-null  int64 
 4   rating        18548 non-null  object
 5   listed_in     18548 non-null  object
 6   description   18548 non-null  object
 7   country_y     18548 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.8+ MB


In [55]:
df['cast'] = df['cast'].str.split(',')

In [56]:
df['release_year'] = df['release_year'].astype(str).str.split(',')
df['rating'] = df['rating'].str.split(',')
df['listed_in'] = df['listed_in'].str.split(',')
df['country_y'] = df['country_y'].str.split(',')

In [57]:
df['type'] = df['type'].str.split(',')

In [58]:
df.head()

,type,title,cast,release_year,rating,listed_in,description,country_y
0,[Movie],Take Care Good Night,"[Mahesh Manjrekar, Abhay Mahajan, Sachin Khe...",[2018],[PG-13],"[Drama, International]","[a, metro, family, decides, to, fight, a, cybe...",[India]
1,[Movie],Whoop!,[Abish Mathew],[2018],[R],"[Arts, Entertainment, and, Culture]","[abish, mathew, is, the, world's, greatest, st...",[India]
2,[Movie],Wedding Cha Shinema,"[Shivraj Waichal, Rucha Inamdar, Mukta Barve]",[2019],[G],"[Comedy, International]","[an, aspiring, filmmaker, reluctantly, takes, ...",[India]
3,[Movie],Waiting for You,"[Colin Morgan, Fanny Ardant, Abdelkrim Bahloul]",[2018],[PG-13],"[Drama, Romance, Suspense]","[paul, (morgan), investigates, his, late, fath...",[United Kingdom]
4,[Movie],Vows of Deception,"[Cheryl Ladd, Nick Manusco, Nancy Cartwright]",[1996],[PG-13],"[Drama, Suspense]","[when, a, prominent, lawyer, is, found, murder...",[United States]


In [59]:
df['listed_in'] = df['listed_in'].apply(lambda x: [i.replace(' ', '') for i in x])
df['cast'] = df['cast'].apply(lambda x: [i.replace(' ','') for i in x])

In [60]:
df.head()

,type,title,cast,release_year,rating,listed_in,description,country_y
0,[Movie],Take Care Good Night,"[MaheshManjrekar, AbhayMahajan, SachinKhedekar]",[2018],[PG-13],"[Drama, International]","[a, metro, family, decides, to, fight, a, cybe...",[India]
1,[Movie],Whoop!,[AbishMathew],[2018],[R],"[Arts, Entertainment, and, Culture]","[abish, mathew, is, the, world's, greatest, st...",[India]
2,[Movie],Wedding Cha Shinema,"[ShivrajWaichal, RuchaInamdar, MuktaBarve]",[2019],[G],"[Comedy, International]","[an, aspiring, filmmaker, reluctantly, takes, ...",[India]
3,[Movie],Waiting for You,"[ColinMorgan, FannyArdant, AbdelkrimBahloul]",[2018],[PG-13],"[Drama, Romance, Suspense]","[paul, (morgan), investigates, his, late, fath...",[United Kingdom]
4,[Movie],Vows of Deception,"[CherylLadd, NickManusco, NancyCartwright]",[1996],[PG-13],"[Drama, Suspense]","[when, a, prominent, lawyer, is, found, murder...",[United States]


In [61]:
df['listed_in'] = df['listed_in'].replace('and,','')

In [191]:
movie = pd.DataFrame()
movie['title'] = df['title']
movie['tags'] = df['description'] + df['listed_in'] + df['type'] + df['release_year'] + df['rating'] + df['country_y'] + df['cast']

In [192]:
movie['tags'] = movie['tags'].apply(lambda x:' '.join(x))

In [193]:
movie['tags'][0]

'a metro family decides to fight a cyber criminal threatening their stability and pride. Drama International Movie 2018 PG-13 India MaheshManjrekar AbhayMahajan SachinKhedekar'

In [194]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [195]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def clean_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Convert to lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Join tokens back into a single string
    cleaned_text = ' '.join(tokens)

    return cleaned_text

movie['tags'] = movie['tags'].apply(clean_text)

In [196]:
movie['tags'][0]

'metro family decides fight cyber criminal threatening stability pride drama international movie pg india maheshmanjrekar abhaymahajan sachinkhedekar'

In [197]:
movie.head()

,title,tags
0,Take Care Good Night,metro family decides fight cyber criminal thre...
1,Whoop!,abish mathew worlds greatest standup comedian ...
2,Wedding Cha Shinema,aspiring filmmaker reluctantly takes assignmen...
3,Waiting for You,paul morgan investigates late fathers increasi...
4,Vows of Deception,prominent lawyer found murdered wife promptly ...


In [198]:
movie[movie['title'].str.contains("Marvel's")]

,title,tags
3825,Marvel's Hulk: Where Monsters Dwell,halloween hulk dr strange battle monsters wrea...
5147,Marvel's Iron Man & Hulk: Heroes United,scientists join hulks gamma energy tony starks...
6343,Marvel's Iron Man & Captain America: Heroes Un...,iron man captain america battle red skull task...
6466,Marvel's Iron Man & Hulk: Heroes United,avengers iron man hulk team fight energydevour...
14849,Marvel's Agents of S.H.I.E.L.D.,agent phil coulson seen action marvels avenger...
15446,Marvel's Jessica Jones,haunted traumatic past jessica jones uses gift...
15573,Marvel's The Punisher,former marine punish criminals responsible fam...
15658,Marvel's Daredevil,blinded young boy matt murdock fights injustic...
15695,Marvel's Iron Fist,danny rand resurfaces years presumed dead powe...
15736,Marvel's Luke Cage,hoodiewearing unbreakable excon fights clear n...


In [199]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18548 entries, 0 to 19249
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   18548 non-null  object
 1   tags    18548 non-null  object
dtypes: object(2)
memory usage: 950.8+ KB


In [200]:
movie.reset_index(inplace=True,drop = True)

In [201]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

# Content-Based Recommendation System using Latent Semantic Analysis (LSA) and k-nearest neighbors (KNN)

In [202]:
vectorizer = CountVectorizer(stop_words='english')
#tfidf = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(movie['tags'])

In [224]:
num_topics = 15  # Number of topics to generate
lsa = TruncatedSVD(n_components=num_topics)
lsa.fit(X)
topic_distribution = normalize(lsa.transform(X))

In [225]:
import sys
# Calculate the size of topic_distribution in bytes
topic_distribution_size_bytes = sys.getsizeof(topic_distribution)

# Convert to megabytes (MB)
topic_distribution_size_mb = topic_distribution_size_bytes / (1024 * 1024)

print("Size of topic_distribution:")
print("Bytes:", topic_distribution_size_bytes)
print("Megabytes (MB):", topic_distribution_size_mb)

Size of topic_distribution:
Bytes: 2225888
Megabytes (MB): 2.122772216796875


In [226]:
knn_t = NearestNeighbors(metric='euclidean', algorithm='brute')
knn_t.fit(topic_distribution)

NearestNeighbors(algorithm='brute', metric='euclidean')

In [227]:
topic__size_bytes = sys.getsizeof(knn_t)

# Convert to megabytes (MB)
topic__size_mb = topic__size_bytes / (1024 * 1024)

print("Size of topic_:")
print("Bytes:", topic__size_bytes)
print("Megabytes (MB):", topic__size_mb)

Size of topic_:
Bytes: 48
Megabytes (MB): 4.57763671875e-05


In [228]:
def recommend_movies(movie_title, num_recommendations=5):
    # Find the index of the movie title
    movie_index = movie[movie['title'] == movie_title].index[0]

    # Get the indices of similar movies based on content similarity
    _, indices = knn_t.kneighbors([topic_distribution[movie_index]], n_neighbors=num_recommendations+1)

    # Exclude the movie itself from recommendations
    indices = indices.squeeze()[1:]

    # Return the recommended movie titles
    recommended_movies = movie.iloc[indices]['title']
    return recommended_movies

# Example usage
movie_title = "Aladdin (2019)"  # Replace with the movie title you want recommendations for
recommendations = recommend_movies(movie_title)
print("Recommended Movies:")
print(recommendations)

Recommended Movies:
6043          The Nutcracker and the Four Realms
5963                                  Maleficent
6129                                 John Carter
6130                            Bride of Boogedy
6610    The Chronicles of Narnia: Prince Caspian
Name: title, dtype: object


# Content-Based Recommendation System with Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(X)

In [ ]:
import sys
# Calculate the size of topic_distribution in bytes
similarity_size_bytes = sys.getsizeof(similarity)

# Convert to megabytes (MB)
similarity_size_mb = similarity_size_bytes / (1024 * 1024)

print("Size of similarity:")
print("Bytes:", similarity_size_bytes)
print("Megabytes (MB):", similarity_size_mb)

Size of similarity:
Bytes: 2752226560
Megabytes (MB): 2624.727783203125


In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]  ## Creating distances with index position and fetching out 5 top

[(3981, 0.2034787998649838),
 (4971, 0.16863471508944566),
 (2577, 0.12398594322201954),
 (4464, 0.11970303889031969),
 (4841, 0.11453929331561093)]

In [ ]:
def recommend(movie_data, movie_title):
    movie_index = movie_data[movie_data['title'] == movie_title].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(movie.iloc[i[0]].title)

In [ ]:
movie.head()

,title,tags
0,Take Care Good Night,a metro family decides to fight a cyber crimin...
1,Whoop!,abish mathew is the worlds greatest standup co...
2,Wedding Cha Shinema,an aspiring filmmaker reluctantly takes up the...
3,Waiting for You,paul morgan investigates his late fathers incr...
4,Vows of Deception,when a prominent lawyer is found murdered his ...


In [ ]:
recommend(movie,"X-Men: First Class")

X-Men: Days of Future Past
X-Men: Dark Phoenix
X-Men: Apocalypse
X-Men
X-Men Origins: Wolverine


In [ ]:
movie.head()

,title,tags
0,Take Care Good Night,a metro family decides to fight a cyber crimin...
1,Whoop!,abish mathew is the worlds greatest standup co...
2,Wedding Cha Shinema,an aspiring filmmaker reluctantly takes up the...
3,Waiting for You,paul morgan investigates his late fathers incr...
4,Vows of Deception,when a prominent lawyer is found murdered his ...


# Content-Based Recommendation System Using KNN and Cosine Similarity

In [133]:
content_similarity = cosine_similarity(X, X)

In [ ]:
svd = TruncatedSVD(n_components=100, random_state=42)
content_similarity_reduced = svd.fit_transform(content_similarity)

# Normalize the reduced similarity matrix
content_similarity_reduced = normalize(content_similarity_reduced)

# Calculate the new size of the reduced similarity matrix
bytes_size = content_similarity_reduced.nbytes
mb_size = bytes_size / (1024 * 1024)

print("Size of content_similarity_reduced:")
print("Bytes:", bytes_size)
print("Megabytes (MB):", mb_size)

Size of content_similarity_reduced:
Bytes: 1483840
Megabytes (MB): 1.41510009765625


In [ ]:
import sys
# Calculate the size of topic_distribution in bytes
content_similarity_size_bytes = sys.getsizeof(content_similarity)

# Convert to megabytes (MB)
content_similarity_size_mb = content_similarity_size_bytes / (1024 * 1024)

print("Size of content_similarity:")
print("Bytes:", content_similarity_size_bytes)
print("Megabytes (MB):", content_similarity_size_mb)

Size of content_similarity:
Bytes: 2752226560
Megabytes (MB): 2624.727783203125


In [214]:
knn = NearestNeighbors(metric='euclidean', algorithm='brute')
knn.fit(content_similarity)

NearestNeighbors(algorithm='brute', metric='cosine')

In [215]:
def recommend_movies(movie_title, num_recommendations=5):
    # Find the index of the movie in the dataframe
    movie_index = movie[movie['title'] == movie_title].index[0]

    # Get the indices of similar movies based on content similarity
    _, indices = knn.kneighbors([content_similarity[movie_index]], n_neighbors=num_recommendations+1)

    # Exclude the movie itself from recommendations
    indices = indices.squeeze()[1:]

    # Return the recommended movie titles
    recommended_movies = movie.iloc[indices]['title']
    return recommended_movies

In [ ]:
movie['title']

0                             Take Care Good Night
1                                           Whoop!
2                              Wedding Cha Shinema
3                                  Waiting for You
4                                Vows of Deception
                           ...                    
18543       Marvel Studios: Expanding the Universe
18544      Marvel's Avengers: Secret Wars (Shorts)
18545    Marvel's Guardians of the Galaxy (Shorts)
18546             Marvel's Rocket & Groot (Shorts)
18547                    Star Wars Rebels (Shorts)
Name: title, Length: 18548, dtype: object

In [216]:
recommended_movies = recommend_movies("Aladdin (2019)", num_recommendations=5)
print(recommended_movies)

6043    The Nutcracker and the Four Realms
6330                                 Dumbo
6127          Maleficent: Mistress of Evil
6130                      Bride of Boogedy
6129                           John Carter
Name: title, dtype: object


Considering computational efficiency and performance, we have opted for a Content-Based Recommendation System that leverages Latent Semantic Analysis (LSA) and k-nearest neighbors (KNN). This approach ensures smoother and more efficient operations in the recommendation process. By utilizing LSA, we extract underlying patterns and topics from the movie tags, while KNN allows us to identify similar movies based on these latent features. This combination of LSA and KNN strikes a balance between computational resources and recommendation accuracy, resulting in an effective and performant system.

 Saving The Files For Further Development for ImDb App

In [229]:
import pickle
pickle.dump(movie,open('recommend/movie_list.pkl','wb'))
pickle.dump(topic_distribution,open('recommend/similarity.pkl','wb'))
pickle.dump(knn_t,open('recommend/knn.pkl','wb'))